In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [ ]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from keras.activations import relu
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from sklearn import svm
from keras.preprocessing import image
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os
import cv2
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as pr
from sklearn.preprocessing import LabelEncoder 
from sklearn import preprocessing
from glob import glob 

In [ ]:
train = '/kaggle/input/plant-seedlings-classification/train/*/*.png'
test = '/kaggle/input/plant-seedlings-classification/test/*.png'

In [ ]:
data = glob(train)

x=[]
y=[]

data_count= len(data)

print("Reading Data")
for d in data:
    x.append(cv2.resize(cv2.imread(d),(244,244)))
    y.append(d.split('/')[5])
print("Data Read Complete")
x = np.asarray(x)
y = pd.DataFrame(y)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
plt.imshow(x[0])

In [ ]:
num = 18
num_row = 3
num_col = 6                              # plot images
fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
for i in range(num):
    ax = axes[i//num_col, i%num_col]
    ax.imshow(x[i], cmap='gray')
    #ax.set_title('{}'.format(y_train[i]))
plt.show()

In [ ]:
print(y.shape)
y[0].unique()
#print(x[1][1])

In [ ]:
def convert_images(x):
    X = np.array(x)          
    # X = X.reshape(X.shape[0],100*100*3)            ## reshape 3D to 1D vector 
    return X

def convert_labels(y):
    enc = LabelEncoder()                        
    P = enc.fit_transform(y)                ## convert string labels to numbers 
    Y = to_categorical(P)                        ## convert number to one-hot-encode form 
    return Y
X = np.array(x)/255.0 
Y = convert_labels(y)

#print(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
X_train = X_train.astype('float32')    #chang integer -> float
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print("Shape of training data:")
print(X_train.shape)    #50,000 ภาพ ขนาด 32*32  3npg
print(Y_train.shape)
print("Shape of test data:")
print(X_test.shape)
print(Y_test.shape)

In [ ]:
cnn = Sequential()

cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(190, 190, 3)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(256, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))

cnn.add(Flatten())        

cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(144, activation='relu'))
cnn.add(Dense(12, activation='softmax'))

sgd = (learning_rate=0.01)

cnn.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)
print(cnn.summary())


In [ ]:
import matplotlib.pyplot as plt

history = cnn.fit(X_train, Y_train, validation_split=0.1,  epochs=5)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('CNN model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
score = cnn.evaluate(X_test, Y_test, verbose=0)     #วัดความแม่นยำ
# y= cnn.predict(img)
print(cnn.metrics_names)
print(score)

In [ ]:
test_path = test
info = glob(test_path)
#creating two list for isnseritng testing and testing labels
test_img = []
test_y = []
num = len(info)
#print(num)
print(info)
for i in info:
    test_y.append(i.split('/')[-1])
    test_img.append(cv2.resize(cv2.imread(i),(190,190)))
test_img= np.asarray(test_img)

In [ ]:
# Normalization of the Image Data
test_img = test_img.astype('float32')
test_img /= 255

In [ ]:
# #using the model to predict the values on the testing data
prediction = cnn.predict(test_img)
print(prediction.shape)
# TEST = []
# for i in range(len(prediction)):
#     pred = np.argmax(prediction[i])
#     TEST.append(list(label_map.keys())[list(label_map.values()).index(pos)])

# #print(prediction[0])
#p_test = cnn.predict(X_test, verbose=1)

pred = np.argmax(prediction,axis=1)
print(pred)

# # out_df = pd.DataFrame({'file':filenames , 
# #                        'species': labelEncoder(pred)})
# # out_df.to_csv('submission.csv', index=False)
# predStr = LabelEncoder.classes_[pred]
# submission = {'file':test_y,'species':predStr}
# submission = pd.DataFrame(submission)
# submission.to_csv("Prediction.csv",index=False)

# # submission['target'] = cnn.predict_proba(test)[:,1]
# # submission.to_csv('submission.csv', index=False)

# sample = pd.read_csv('../input/plant-seedlings-classification/sample_submission.csv')
# sample['species'] = TEST
# sample

In [ ]:
def category (label):
    if label == 0: return  'Black-grass'
    elif label == 1: return 'Charlock'
    elif label == 2: return 'Cleavers'
    elif label == 3: return 'Common Chickweed'
    elif label == 4: return 'Common wheat'
    elif label == 5: return 'Fat Hen'
    elif label == 6: return 'Loose Silky-bent'
    elif label == 7: return 'Maize'
    elif label == 8: return 'Scentless Mayweed'
    elif label == 9: return 'Shepherds Purse'
    elif label == 10: return 'Small-flowered Cranesbill'
    elif label == 11: return 'Sugar beet'

In [ ]:
pred_cate=[]
for label in pred :
    pred_category= category(label)
    pred_cate.append(pred_category)
pred_cate

In [ ]:
sample = pd.read_csv('../input/plant-seedlings-classification/sample_submission.csv')
sample['species'] = pred_cate
sample.to_csv("submission.csv", index=False)
sample.head()